In [ ]:
from igraph import *
from collections import deque
import random
import math

In [ ]:
#returns maximum community label
def get_max_label(g):
    maxVertex = max(g.vs, key = lambda v: max(v["communities"]) if len(v["communities"]) > 0 else 0)
    maxLabel = max(maxVertex["communities"])
    return maxLabel

In [ ]:
#removes unused spaces between community labels
def normalize_labels(g):
    maxLabel = get_max_label(g)
    currentLabel = 0
    for i in range(maxLabel):
        vertices = g.vs.select(lambda v: i in v["communities"])
        if len(vertices) > 0 and i != currentLabel:
            #remap vertices
            for v in g.vs:
                for idx in v["communities"]:
                    if (idx == i):
                        idx = currentLabel
            
            currentLabel += 1    

In [ ]:
#plots full graph
def plot_full(g, filename="output.svg", spread = 1):
    print("Plotting graph with " + str(len(g.vs)) + " vertices and " + str(len(g.es)) + " edges, begin...")
    normalize_labels(g)
    
    #get max community label
    maxLabel = get_max_label(g)
    
    #define palette
    palette = RainbowPalette(maxLabel + 1)
                   
    #map communities to colors
    for v in g.vs:
        if (len(v["communities"]) == 0):
            v["color"] = (1.0, 1.0, 1.0, 1.0)
        else:            
            v["color"] = palette.get(v["communities"][0])
        
        label = ""
        for i in v["communities"]:
            label += str(i) + " "
            
        if len(label) > 0:
            label = label[:-1]
            
        if "name" in v.attributes():
            v["label"] = v["name"]
            if len(label) > 0:
                v["label"] +=  "\n(" + label + ")"
                
            v["size"] = len(v["name"]) * 8
        else:
            v["label"] = label
            v["size"] = (len(v["communities"]) + 1) * 12 + 5
                   
    #plot
    visual_style = {}
    verticesCount = len(g.vs)
    if (verticesCount > 100):
        layout = g.layout("lgl")
        visual_style["layout"] = layout
        
    if (any("name" in v.attributes() for v in g.vs)):
        spread *= 2.7
        
    scaleMultiplier = math.sqrt(verticesCount) * spread * 6
    visual_style["bbox"] = (16 * (scaleMultiplier + 20), 9 * (scaleMultiplier + 20))
    visual_style["margin"] = 20
    plot(g, filename, **visual_style)
    print("Plotting sucessful")

In [ ]:
#plots all vertices with distance from start_idx lower than depth
def plot_bfs(g, start_idx = 0, depth = 10, filename="output.svg", spread = 1):
    print("Running bfs with " + str(depth) + " depth, " + str(len(g.vs)) + " vertices and " + str(len(g.es)) + " edges, begin...")
    all_vertices = []
    vertices = deque()
    vertices.append(g.vs[start_idx])
    
    for v in g.vs:
        v["processed"] = False
        v["distance"] = len(g.vs)
        
    g.vs[start_idx]["distance"] = 0
    
    while (len(vertices) > 0):
        current = vertices.popleft()
        current["processed"] = True
        if current["distance"] == depth:
            break
            
        all_vertices.append(current)
        edges = current.all_edges()
        for e in edges:
            if (e.target_vertex["processed"] == False):
                e.target_vertex["distance"] = min(e.target_vertex["distance"], current["distance"] + 1)
                vertices.append(e.target_vertex)
            
    subgraph = g.subgraph(all_vertices)
    print("Bfs sucessful")
    plot_full(subgraph, filename, spread)